In [2]:
# Load displacement data
from pathlib import Path

import pandas as pd
import geopandas as gpd

from climada.util.constants import DEF_CRS

# Mapping from data columns to exposure regions

# Load the true data
# Create the data frame for the true impact data
DISPLACEMENT_DATA_PATH = Path("../data/displacement_data/PDMA_report_sep_17.xlsx")
displacement_data = pd.read_excel(DISPLACEMENT_DATA_PATH)

# Load the Pakistan shapefile
SHAPEFILE_PATH = Path(
    "../data/boundaries/pak_adm_wfp_20220909_shp/pak_admbnda_adm2_wfp_20220909.shp"
)
shapefile_pakistan = gpd.read_file(SHAPEFILE_PATH).to_crs(DEF_CRS)

# Carve out Sindh provinces
sindh_provinces = shapefile_pakistan.loc[shapefile_pakistan["ADM1_EN"] == "Sindh"]


## Load exposures

In [3]:
from pathlib import Path

import shapely

from climada.entity import Exposures
from climada.util.constants import DEF_CRS

# Load WorldPop
WORLD_POP_PAK_PATH = Path("../data/population/pak_ppp_2020_UNadj_constrained.tif")
#WORLD_POP_PAK_PATH = Path("../data/population/pak_ppp_2020_UNadj.tif")
exposure = Exposures.from_raster(
    WORLD_POP_PAK_PATH,
    dst_crs=DEF_CRS,
    geometry=[shapely.union_all(sindh_provinces["geometry"])],
)
exposure.set_geometry_points()
exposure.gdf = exposure.gdf.loc[exposure.gdf["value"] > 0, :]  # Only retain values > 0
exposure.gdf["impf_FL"] = 1

In [4]:
# Assign Admin3 regions
gdf_joined = exposure.gdf.sjoin(sindh_provinces, how="left", predicate="within")

# Merge Karachi regions
adm2 = gdf_joined["ADM2_EN"]
adm2[adm2.str.contains("Karachi")] = "Karachi"
gdf_joined["ADM2_EN"] = adm2

# Set new gdf
exposure.set_gdf(gdf_joined)

In [5]:
exposure.gdf

,longitude,latitude,value,geometry,impf_FL,index_right,Shape_Leng,Shape_Area,ADM2_EN,ADM2_PCODE,ADM2_REF,ADM2ALT1EN,ADM2ALT2EN,ADM1_EN,ADM1_PCODE,ADM0_EN,ADM0_PCODE,date,validOn,validTo
46977,69.510000,28.478333,55.389271,POINT (69.51000 28.47833),1,140,2.880124,0.240492,Kashmore,PK710,NaN,NaN,NaN,Sindh,PK7,Pakistan,PK,2022-09-02,2022-09-09,NaN
46978,69.510833,28.478333,51.370152,POINT (69.51083 28.47833),1,140,2.880124,0.240492,Kashmore,PK710,NaN,NaN,NaN,Sindh,PK7,Pakistan,PK,2022-09-02,2022-09-09,NaN
52409,69.507500,28.477500,54.412155,POINT (69.50750 28.47750),1,140,2.880124,0.240492,Kashmore,PK710,NaN,NaN,NaN,Sindh,PK7,Pakistan,PK,2022-09-02,2022-09-09,NaN
52410,69.508333,28.477500,54.479439,POINT (69.50833 28.47750),1,140,2.880124,0.240492,Kashmore,PK710,NaN,NaN,NaN,Sindh,PK7,Pakistan,PK,2022-09-02,2022-09-09,NaN
52411,69.509166,28.477500,57.147064,POINT (69.50917 28.47750),1,140,2.880124,0.240492,Kashmore,PK710,NaN,NaN,NaN,Sindh,PK7,Pakistan,PK,2022-09-02,2022-09-09,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28909952,67.585000,24.052500,139.486984,POINT (67.58500 24.05250),1,152,6.703949,0.772314,Sujawal,PK722,NaN,NaN,NaN,Sindh,PK7,Pakistan,PK,2022-09-02,2022-09-09,NaN
28909953,67.585833,24.052500,140.509171,POINT (67.58583 24.05250),1,152,6.703949,0.772314,Sujawal,PK722,NaN,NaN,NaN,Sindh,PK7,Pakistan,PK,2022-09-02,2022-09-09,NaN
28909954,67.586666,24.052500,140.043671,POINT (67.58667 24.05250),1,152,6.703949,0.772314,Sujawal,PK722,NaN,NaN,NaN,Sindh,PK7,Pakistan,PK,2022-09-02,2022-09-09,NaN
29736377,67.839166,23.925833,73.159149,POINT (67.83917 23.92583),1,152,6.703949,0.772314,Sujawal,PK722,NaN,NaN,NaN,Sindh,PK7,Pakistan,PK,2022-09-02,2022-09-09,NaN


## Load Hazard

In [6]:
from climada.hazard import Hazard
from rasterio.warp import Resampling

HAZ_TYPE ='FL'
HAZ_FILE = '../data/flood_depth/sindh_flooddepth1.tif'

hazard = Hazard.from_raster(HAZ_FILE, 
                            haz_type=HAZ_TYPE,
                            transform=exposure.meta['transform'],
                            height=exposure.meta['height'],
                            width=exposure.meta['width'],
                            resampling=Resampling.bilinear)

#### Define function for impact function

In [9]:
from climada.entity import ImpactFunc, ImpactFuncSet

def impf_set_step(threshold, mdd_max):
    impf_step = ImpactFunc.from_step_impf(intensity=(0,threshold,10),
                                          mdd=(0, mdd_max),
                                        haz_type='FL')
    impf_step.name = 'FL sigmoid func'
    impf_step.intensity_unit = 'm'

    impf_set = ImpactFuncSet()
    impf_set.append(impf_step)
    
    return impf_set

# calibrate function for each district

In [ ]:
import numpy as np

from climada.engine import ImpactCalc

thresholds_list = np.arange(0.05, 5.01, 0.01)
imp_loop = None

for thresholds in thresholds_list:
    
    impf_step = impf_set_step(thresholds, mdd_max=1.)
    
    impact = ImpactCalc(exposure, impf_step, hazard).impact()
    imp_reg = impact.impact_at_reg(agg_regions=exposure.gdf["ADM2_EN"]).reset_index(drop=True)
    
    df_threshold = pd.DataFrame({'threshold': thresholds}, index=[0])
    
    if imp_loop is None:
        imp_loop = pd.concat([df_threshold, imp_reg], axis=1)
    else:
        imp_loop = pd.concat([imp_loop, pd.concat([df_threshold, imp_reg], axis=1)], axis=0)
        
imp_loop.reset_index(inplace=True)

In [ ]:
opt_thresholds_per_reg = {}
opt_value ={}

for admin2 in pdma_report.index:
    
    idx = np.argmin(np.abs(imp_loop[admin2].values-pdma_report.T[admin2].values))
    
    print(idx)
    
    opt_thresholds_per_reg[admin2] = imp_loop['threshold'][idx]
    opt_value[admin2] = imp_loop[admin2][idx]

In [ ]:
opt_value

In [ ]:
opt_thresholds_per_reg

In [ ]:
df_urban = pd.read_excel('../exploratory_stats/Sindh_displacement_stats_manual.xlsx')
df_urban.sort_values(by='Sindh_admin2', inplace=True)
df_urban['opt_thresh'] = opt_thresholds_per_reg.values()
df_urban['opt_value'] = opt_value.values()

In [ ]:
df_urban.sort_values(by='pct_flooded_area_urban', inplace=True)

In [ ]:
df_urban.plot.scatter(x='Sindh_admin2', y='opt_thresh')

In [ ]:
df_urban.corr().to_excel('../exploratory_stats/correlation_table.xlsx')

In [ ]:
df_urban_func_plot = df_urban.sort_values(by='pct_flooded_area_urban')
df_urban_func_plot.to_excel('../exploratory_stats/inc_vulnerability_thresholds.xlsx')

In [ ]:
df_urban_func_plot

In [ ]:
opt_imp_thresh = 0.67
imp_thresh_range = [0.368, 1.102]

In [ ]:
legend_element_vulnerability = [Line2D([0], [0], marker='o', color='w', markerfacecolor='#1f77b4',
                                       label='Thresholds fitted inividually \nto each districts in Sindh'),
                                Line2D([0],[0], marker=0, color='w', markeredgecolor='steelblue',
                                       label='Numerically calibrated threshold \nfor the impact step function'),
                                Patch(facecolor='lightblue', edgecolor='w', alpha=.9,
                                      label='Calibration uncertainty')]

In [ ]:
bar_width=0.5

fig = plt.figure(figsize=(12,5))
plt.grid(visible=True, lw=.5, alpha=.5)
plt.grid(which='minor', axis='y',
         lw=.3, alpha=.3)

plt.scatter(np.arange(len(df_urban_func_plot)), df_urban_func_plot['opt_thresh'], marker='o')

plt.axvspan(23.75, 25.2, color='lightgrey', alpha=0.65, lw=0)
plt.bar(24.5,
        height=imp_thresh_range[1]-imp_thresh_range[0],
        bottom=imp_thresh_range[0],
        width=bar_width,
        color='lightblue', alpha=.9)
plt.scatter(24.5,
            opt_imp_thresh,
            color='steelblue',
            marker=0)
plt.scatter(24.5,
            opt_imp_thresh,
            color='steelblue',
            marker=1)

## plot v lines for urban area classification
plt.vlines(np.array([5.5, 20.5]), -0.2, 5.9,
           color='k', lw=.7, linestyle='--')
plt.vlines(23.75, -0.2, 5.9,
           color='k', lw=1.)

plt.xlim([-0.7, 25.2])
plt.ylim([-0.2, 5.9])
plt.ylabel('Flood depth threshold (m)')
plt.xticks(np.append(np.arange(len(df_plot_sig)), [24.5]), 
           labels=np.append(df_urban_func_plot['Sindh_admin2'].to_numpy(), ['Impact Func. Calib. for Sindh']), rotation=90)
plt.legend(handles=legend_element_vulnerability, ncol=3,  loc='upper center')
#plt.hlines(100, -0.7, 24.7, linestyle='--', lw=.7, color='k')

In [ ]:
import numpy as np
from scipy import stats

def pearsonr_ci(x,y,alpha=0.05):
    ''' calculate Pearson correlation along with the confidence interval using scipy and numpy
    Parameters
    ----------
    x, y : iterable object such as a list or np.array
      Input for correlation calculation
    alpha : float
      Significance level. 0.05 by default
    Returns
    -------
    r : float
      Pearson's correlation coefficient
    pval : float
      The corresponding p value
    lo, hi : float
      The lower and upper bound of confidence intervals
    '''

    r, p = stats.pearsonr(x,y)
    r_z = np.arctanh(r)
    se = 1/np.sqrt(x.size-3)
    z = stats.norm.ppf(1-alpha/2)
    lo_z, hi_z = r_z-z*se, r_z+z*se
    lo, hi = np.tanh((lo_z, hi_z))
    return r, p, lo, hi

In [ ]:
corr_test = pearsonr_ci(df_urban_func_plot['pct_flooded_area_urban'], df_urban_func_plot['opt_thresh'])

In [ ]:
corr_test